# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Necessary Library**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline  
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


import warnings
warnings.filterwarnings("ignore")


**Data Loading**

In [2]:
train_data= pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
test_data=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")
train_data.head()

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase,Made_Purchase
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,...,1.0,9.0,3.0,Returning_Visitor,Not Specified,Deny,Not Specified,Other,0.0,False
1,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,...,2.0,2.0,4.0,Returning_Visitor,Female,Deny,Others,Married,0.0,False
2,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,...,3.0,1.0,4.0,Returning_Visitor,Female,ALL,Others,Married,1.0,False
3,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.4,...,4.0,3.0,3.0,Returning_Visitor,Male,ALL,Diploma,Single,0.0,False
4,1.0,0.0,0.0,0.0,0.0,0.000000,0.20,0.20,0.0,0.0,...,2.0,1.0,5.0,Returning_Visitor,Male,Deny,Diploma,Other,1.0,False


In [3]:
train_data.describe()


,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,OS,SearchEngine,Zone,Type of Traffic,WeekendPurchase
count,14578.000000,14581.000000,14578.000000,14596.000000,14608.000000,14564.000000,14580.000000,14602.000000,14599.000000,14581.000000,14597.000000,14609.000000,14614.000000,14588.000000,14610.000000
mean,2.250240,79.300762,0.490739,33.455943,31.559488,1184.346084,0.023366,0.044664,4.812620,0.064083,2.122422,2.356629,3.155673,4.090143,0.234155
std,3.288042,179.374699,1.252376,140.146256,44.897089,2009.496307,0.050011,0.049912,16.887366,0.202583,0.914404,1.721823,2.405155,4.040147,0.423484
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,173.187500,0.000000,0.014502,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000,0.000000
50%,1.000000,5.000000,0.000000,0.000000,17.500000,584.333333,0.003478,0.026406,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000,0.000000
75%,3.000000,91.000000,0.000000,0.000000,38.000000,1434.255128,0.018182,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000,0.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000,1.000000


In [4]:
train_data.columns

Index(['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
       'GoogleMetric:Page Values', 'SeasonalPurchase',
       'Month_SeasonalPurchase', 'OS', 'SearchEngine', 'Zone',
       'Type of Traffic', 'CustomerType', 'Gender', 'Cookies Setting',
       'Education', 'Marital Status', 'WeekendPurchase', 'Made_Purchase'],
      dtype='object')

In [5]:
train_data. dtypes

HomePage                           float64
HomePage_Duration                  float64
LandingPage                        float64
LandingPage_Duration               float64
ProductDescriptionPage             float64
ProductDescriptionPage_Duration    float64
GoogleMetric:Bounce Rates          float64
GoogleMetric:Exit Rates            float64
GoogleMetric:Page Values           float64
SeasonalPurchase                   float64
Month_SeasonalPurchase              object
OS                                 float64
SearchEngine                       float64
Zone                               float64
Type of Traffic                    float64
CustomerType                        object
Gender                              object
Cookies Setting                     object
Education                           object
Marital Status                      object
WeekendPurchase                    float64
Made_Purchase                         bool
dtype: object

**Filling Missing Value**

In [6]:
train_data.replace(' ',np.nan,inplace=True)
test_data.replace(' ',np.nan,inplace=True)
train_data.isnull().sum()

HomePage                           153
HomePage_Duration                  150
LandingPage                        153
LandingPage_Duration               135
ProductDescriptionPage             123
ProductDescriptionPage_Duration    167
GoogleMetric:Bounce Rates          151
GoogleMetric:Exit Rates            129
GoogleMetric:Page Values           132
SeasonalPurchase                   150
Month_SeasonalPurchase             144
OS                                 134
SearchEngine                       122
Zone                               117
Type of Traffic                    143
CustomerType                       144
Gender                             145
Cookies Setting                    144
Education                          136
Marital Status                     130
WeekendPurchase                    121
Made_Purchase                        0
dtype: int64

In [7]:
train_data.head()

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,SearchEngine,Zone,Type of Traffic,CustomerType,Gender,Cookies Setting,Education,Marital Status,WeekendPurchase,Made_Purchase
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,...,1.0,9.0,3.0,Returning_Visitor,Not Specified,Deny,Not Specified,Other,0.0,False
1,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,...,2.0,2.0,4.0,Returning_Visitor,Female,Deny,Others,Married,0.0,False
2,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,...,3.0,1.0,4.0,Returning_Visitor,Female,ALL,Others,Married,1.0,False
3,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.4,...,4.0,3.0,3.0,Returning_Visitor,Male,ALL,Diploma,Single,0.0,False
4,1.0,0.0,0.0,0.0,0.0,0.000000,0.20,0.20,0.0,0.0,...,2.0,1.0,5.0,Returning_Visitor,Male,Deny,Diploma,Other,1.0,False


In [8]:
Y=train_data['Made_Purchase']
X=train_data.drop(['Made_Purchase'],axis=1)

In [9]:
X.isnull().sum()

HomePage                           153
HomePage_Duration                  150
LandingPage                        153
LandingPage_Duration               135
ProductDescriptionPage             123
ProductDescriptionPage_Duration    167
GoogleMetric:Bounce Rates          151
GoogleMetric:Exit Rates            129
GoogleMetric:Page Values           132
SeasonalPurchase                   150
Month_SeasonalPurchase             144
OS                                 134
SearchEngine                       122
Zone                               117
Type of Traffic                    143
CustomerType                       144
Gender                             145
Cookies Setting                    144
Education                          136
Marital Status                     130
WeekendPurchase                    121
dtype: int64

In [10]:
X_1_train = X.select_dtypes(include = ['float64'])
X_1_test = test_data.select_dtypes(include = ['float64'])

In [11]:
X_1_test

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,OS,SearchEngine,Zone,Type of Traffic,WeekendPurchase
0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,1.0,1.0,1.0,1.0,0.0
1,0.0,0.000000,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,2.0,2.0,1.0,2.0,0.0
2,0.0,0.000000,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,2.0,2.0,1.0,3.0,0.0
3,0.0,0.000000,0.0,0.0,2.0,37.000000,0.000000,0.100000,0.0,0.8,2.0,2.0,2.0,3.0,0.0
4,0.0,0.000000,0.0,0.0,16.0,407.750000,0.018750,0.025833,0.0,0.4,1.0,1.0,4.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6594,0.0,0.000000,0.0,0.0,7.0,208.000000,0.000000,0.028571,0.0,0.0,4.0,1.0,1.0,5.0,1.0
6595,0.0,0.000000,3.0,44.0,179.0,1738.472529,0.000027,0.025998,0.0,0.0,2.0,4.0,9.0,11.0,0.0
6596,5.0,99.166667,1.0,27.0,33.0,NaN,0.002778,0.009127,0.0,0.6,8.0,5.0,1.0,2.0,0.0
6597,0.0,0.000000,0.0,0.0,3.0,9.000000,0.066667,0.133333,0.0,0.0,2.0,2.0,2.0,3.0,1.0


**Imputing using Simple Imputer**

In [12]:
si=SimpleImputer(missing_values=np.nan,strategy='mean')
X1_train=si.fit_transform(X_1_train)
X1_test=si.fit_transform(X_1_test)

In [13]:
X1_train

array([[ 0. ,  0. ,  0. , ...,  9. ,  3. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  2. ,  4. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  1. ,  4. ,  1. ],
       ...,
       [ 2. , 75.6,  2. , ...,  4. ,  2. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  4. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  3. ,  1. ,  0. ]])

In [14]:
df1_train=pd.DataFrame(X1_train,columns=['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
       'GoogleMetric:Page Values', 'SeasonalPurchase',
        'OS', 'SearchEngine', 'Zone',
       'Type of Traffic', 'WeekendPurchase'])

df1_test=pd.DataFrame(X1_test,columns=['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
       'GoogleMetric:Page Values', 'SeasonalPurchase',
        'OS', 'SearchEngine', 'Zone',
       'Type of Traffic', 'WeekendPurchase'])


In [15]:
df1_test

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,OS,SearchEngine,Zone,Type of Traffic,WeekendPurchase
0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,1.0,1.0,1.0,1.0,0.0
1,0.0,0.000000,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,2.0,2.0,1.0,2.0,0.0
2,0.0,0.000000,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,2.0,2.0,1.0,3.0,0.0
3,0.0,0.000000,0.0,0.0,2.0,37.000000,0.000000,0.100000,0.0,0.8,2.0,2.0,2.0,3.0,0.0
4,0.0,0.000000,0.0,0.0,16.0,407.750000,0.018750,0.025833,0.0,0.4,1.0,1.0,4.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6594,0.0,0.000000,0.0,0.0,7.0,208.000000,0.000000,0.028571,0.0,0.0,4.0,1.0,1.0,5.0,1.0
6595,0.0,0.000000,3.0,44.0,179.0,1738.472529,0.000027,0.025998,0.0,0.0,2.0,4.0,9.0,11.0,0.0
6596,5.0,99.166667,1.0,27.0,33.0,1151.231143,0.002778,0.009127,0.0,0.6,8.0,5.0,1.0,2.0,0.0
6597,0.0,0.000000,0.0,0.0,3.0,9.000000,0.066667,0.133333,0.0,0.0,2.0,2.0,2.0,3.0,1.0


In [16]:
X_2_train = X.select_dtypes(include = ['object'])
X_2_test = test_data.select_dtypes(include = ['object'])

In [17]:
X_2_test

,Month_SeasonalPurchase,CustomerType,Gender,Cookies Setting,Education,Marital Status
0,Feb,Returning_Visitor,Not Specified,Deny,Not Specified,Single
1,Feb,Returning_Visitor,Not Specified,ALL,Graduate,Married
2,Feb,Returning_Visitor,Female,Required,Diploma,Other
3,Feb,Returning_Visitor,Not Specified,Required,Graduate,Other
4,Feb,Returning_Visitor,Female,Deny,Others,Other
...,...,...,...,...,...,...
6594,Feb,Returning_Visitor,Not Specified,Required,Graduate,Other
6595,Aug,Returning_Visitor,Male,Required,Not Specified,Other
6596,May,Returning_Visitor,Female,Required,Graduate,Married
6597,May,Returning_Visitor,Not Specified,Deny,Diploma,Other


In [18]:
si=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
X2_train=si.fit_transform(X_2_train)
X2_test=si.fit_transform(X_2_test)

In [19]:
X2_test

array([['Feb', 'Returning_Visitor', 'Not Specified', 'Deny',
        'Not Specified', 'Single'],
       ['Feb', 'Returning_Visitor', 'Not Specified', 'ALL', 'Graduate',
        'Married'],
       ['Feb', 'Returning_Visitor', 'Female', 'Required', 'Diploma',
        'Other'],
       ...,
       ['May', 'Returning_Visitor', 'Female', 'Required', 'Graduate',
        'Married'],
       ['May', 'Returning_Visitor', 'Not Specified', 'Deny', 'Diploma',
        'Other'],
       ['May', 'Returning_Visitor', 'Male', 'Required', 'Diploma',
        'Single']], dtype=object)

In [20]:
df2_train=pd.DataFrame(X2_train,columns=['Month_SeasonalPurchase','CustomerType', 'Gender', 'Cookies Setting','Education', 'Marital Status'])
df2_test=pd.DataFrame(X2_test,columns=['Month_SeasonalPurchase','CustomerType', 'Gender', 'Cookies Setting','Education', 'Marital Status'])

In [21]:
df2_test

,Month_SeasonalPurchase,CustomerType,Gender,Cookies Setting,Education,Marital Status
0,Feb,Returning_Visitor,Not Specified,Deny,Not Specified,Single
1,Feb,Returning_Visitor,Not Specified,ALL,Graduate,Married
2,Feb,Returning_Visitor,Female,Required,Diploma,Other
3,Feb,Returning_Visitor,Not Specified,Required,Graduate,Other
4,Feb,Returning_Visitor,Female,Deny,Others,Other
...,...,...,...,...,...,...
6594,Feb,Returning_Visitor,Not Specified,Required,Graduate,Other
6595,Aug,Returning_Visitor,Male,Required,Not Specified,Other
6596,May,Returning_Visitor,Female,Required,Graduate,Married
6597,May,Returning_Visitor,Not Specified,Deny,Diploma,Other


In [22]:
df_train = pd.concat([df1_train, df2_train], axis=1)
df_test = pd.concat([df1_test, df2_test], axis=1)

In [23]:
df_test

,HomePage,HomePage_Duration,LandingPage,LandingPage_Duration,ProductDescriptionPage,ProductDescriptionPage_Duration,GoogleMetric:Bounce Rates,GoogleMetric:Exit Rates,GoogleMetric:Page Values,SeasonalPurchase,...,SearchEngine,Zone,Type of Traffic,WeekendPurchase,Month_SeasonalPurchase,CustomerType,Gender,Cookies Setting,Education,Marital Status
0,0.0,0.000000,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,...,1.0,1.0,1.0,0.0,Feb,Returning_Visitor,Not Specified,Deny,Not Specified,Single
1,0.0,0.000000,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,...,2.0,1.0,2.0,0.0,Feb,Returning_Visitor,Not Specified,ALL,Graduate,Married
2,0.0,0.000000,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,...,2.0,1.0,3.0,0.0,Feb,Returning_Visitor,Female,Required,Diploma,Other
3,0.0,0.000000,0.0,0.0,2.0,37.000000,0.000000,0.100000,0.0,0.8,...,2.0,2.0,3.0,0.0,Feb,Returning_Visitor,Not Specified,Required,Graduate,Other
4,0.0,0.000000,0.0,0.0,16.0,407.750000,0.018750,0.025833,0.0,0.4,...,1.0,4.0,3.0,0.0,Feb,Returning_Visitor,Female,Deny,Others,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6594,0.0,0.000000,0.0,0.0,7.0,208.000000,0.000000,0.028571,0.0,0.0,...,1.0,1.0,5.0,1.0,Feb,Returning_Visitor,Not Specified,Required,Graduate,Other
6595,0.0,0.000000,3.0,44.0,179.0,1738.472529,0.000027,0.025998,0.0,0.0,...,4.0,9.0,11.0,0.0,Aug,Returning_Visitor,Male,Required,Not Specified,Other
6596,5.0,99.166667,1.0,27.0,33.0,1151.231143,0.002778,0.009127,0.0,0.6,...,5.0,1.0,2.0,0.0,May,Returning_Visitor,Female,Required,Graduate,Married
6597,0.0,0.000000,0.0,0.0,3.0,9.000000,0.066667,0.133333,0.0,0.0,...,2.0,2.0,3.0,1.0,May,Returning_Visitor,Not Specified,Deny,Diploma,Other


In [24]:
le=LabelEncoder()
y_tra_train=le.fit_transform(Y)
y_tra_train

array([0, 0, 0, ..., 1, 1, 1])

In [25]:
X_train,X_test,y_train,y_test=train_test_split(df_train,y_tra_train,test_size=0.2,random_state=42,shuffle=True)

**Pipeline**

In [26]:
numeric_transformer = Pipeline(steps = [("scaler", StandardScaler())])

n_cols =['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration','ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates','GoogleMetric:Page Values', 'SeasonalPurchase','OS', 'SearchEngine', 'Zone', 
         'Type of Traffic', 'WeekendPurchase']

categorical_transformer = OneHotEncoder()

c_cols =['Month_SeasonalPurchase','CustomerType', 'Gender', 'Cookies Setting','Education', 'Marital Status']


preprocessor = ColumnTransformer(transformers = [("num", numeric_transformer,n_cols),                        
                     ("cat_ohe",categorical_transformer, c_cols ) ] , 
                     remainder = "passthrough")

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
final_X_test = preprocessor.transform(df_test)

**Model Training**

**Random Forest Classifier**

In [27]:
rf = RandomForestClassifier(random_state = 1200)
rf.fit(X_train,y_train)

print("Test Score:",rf.score(X_test,y_test))
print("Train Score:", rf.score(X_train,y_train))

y_pred_test = rf.predict(X_test) # predictions on validation test data

precision,recall,fscore,support = score(y_test,y_pred_test,average = "macro")
print('F-score : {},'.format(fscore))

Test Score: 0.5205293518832711
Train Score: 0.8462321792260692
F-score : 0.4695003307890666,



**Hyperparameter Tun (for RFC) using RandomSearchCV**

In [28]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 15)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 20)]
max_depth.append(None)#  append the default None in the end
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 4, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,2, 3, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# random grid

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 25, cv = 3, verbose=3, random_state=1200, n_jobs = -1,
                               scoring ='accuracy')


In [29]:
# Fit the random search model
#rf_random.fit(X_train,y_train)
#rf_random.best_params_

**Train the Random Forest classifier using best parameters**

In [30]:
 rf = RandomForestClassifier(n_estimators=77, min_samples_split=5, min_samples_leaf=4,max_features='log2',max_depth=10,bootstrap= False,random_state = 1200)
 rf.fit(X_train, y_train)

print("Test Score:",rf.score(X_test,y_test))
print("Train Score:", rf.score(X_train,y_train))

y_pred_test = rf.predict(X_test) # predictions on validation test data

precision,recall,fscore,support = score(y_test,y_pred_test,average = "macro")
print('F-score : {},'.format(fscore))

Test Score: 0.6857821513403461
Train Score: 0.7064663951120163
F-score : 0.582268641545941,


**Trying Different Model**

**Gradient boost classifier**

In [31]:
gbc = GradientBoostingClassifier(random_state = 1200)  # train a dummy model with default parameter 
gbc.fit(X_train,y_train)

print("Test Score:",gbc.score(X_test,y_test))
print("Train Score:", gbc.score(X_train,y_train))

y_pred_test = gbc.predict(X_test) # predictions on validation test data

precision,recall,fscore,support = score(y_test,y_pred_test,average = "macro")
print('F-score : {},'.format(fscore))

Test Score: 0.6878181201221581
Train Score: 0.6804141208418194
F-score : 0.5939385225523145,


In [32]:
# get an idea of the parameter which gave a test score of 0.68781
gbc.get_params(deep = True)

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 1200,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

**Hyperparameter Tun (for GBC)** 

In [33]:
# # narrow down the hyperparameters with gridsearchcv
grid_param = {
     "n_estimators":[int(x) for x in np.linspace(start = 75, stop = 800, num = 30)],
     "max_depth":[int(x) for x in np.linspace(2,6, num = 5)],
     "learning_rate":[0.01,0.1,1]
 }

RScv = RandomizedSearchCV(n_iter = 10 , estimator = gbc,param_distributions = grid_param,cv=5, verbose=1)

In [34]:
#fit the model
#RScv.fit(X_train,y_train)
#RScv.best_params_

In [35]:
#gbc = GradientBoostingClassifier(n_estimators = 300, max_depth = 3, learning_rate=0.01,random_state = 1200)
#gbc.fit(X_train,y_train)

#print("Test Score:",gbc.score(X_test,y_test))
#print("Train Score:", gbc.score(X_train,y_train))

#y_pred_test = gbc.predict(X_test) # predictions on validation test data

#precision,recall,fscore,support = score(y_test,y_pred_test,average = "macro")
#print('F-score : {},'.format(fscore))

#Getting lower score after hyperparamter tuning

**XGboost classifier T*he Final Model With Highest Score***

In [36]:
xgb = XGBClassifier(random_state = 1200) # run a dummy model with default parameters
xgb.fit(X_train,y_train)

print("Test Score:",xgb.score(X_test,y_test))
print("Train Score:", xgb.score(X_train,y_train))

y_pred_test = xgb.predict(X_test) # predictions on validation test data

precision,recall,fscore,support = score(y_test,y_pred_test,average = "macro")
print('F-score : {},'.format(fscore))


Test Score: 0.5951815405497116
Train Score: 0.7721486761710794
F-score : 0.5206571896220158,


**tune hyperparameters for xgboost**

In [37]:
 #param_test1 = {
 #    'learning_rate':[0.01, 0.1, 1, 10],
 #     'n_estimators' : [int(x) for x in np.linspace(start = 100, stop = 1000, num = 7)],
 #     'max_depth': [3,4,5,7,9,10],
 #     'min_child_weight':range(1,6,2)
 #}
# gsearch1 = RandomizedSearchCV(estimator = xgb,n_iter = 100, param_distributions = param_test1,cv =2 , verbose = 2)

#gsearch1.fit(X_train,y_train)

In [38]:
#gsearch1.best_params_

In [39]:
xgb = XGBClassifier(learning_rate =0.01, n_estimators=250, max_depth = 3,min_child_weight= 1,random_state = 42) 
# train the model using best hyperparameters
xgb.fit(X_train,y_train)

print("Test Score:",xgb.score(X_test,y_test))
print("Train Score:", xgb.score(X_train,y_train))

y_pred_test = xgb.predict(X_test) # predictions on validation test data

precision,recall,fscore,support = score(y_test,y_pred_test,average = "macro")
print('F-score : {},'.format(fscore))


Test Score: 0.6868001357312521
Train Score: 0.6688730482009504
F-score : 0.5961870803606832,


**Submission**

In [40]:
predictions = xgb.predict(final_X_test)  # predictions on the test dataset
submission = pd.DataFrame(columns=["id", "Made_Purchase"])  # create an empty dataframe with two columns namely "id" and "Made_Purchase"
submission["id"] = [i for i in range(len(predictions))]  # id is nothing just serial number
submission["Made_Purchase"] = predictions
submission["Made_Purchase"].replace(to_replace=1,value='TRUE',inplace=True)
submission["Made_Purchase"].replace(to_replace=0,value='FALSE',inplace=True)

submission.to_csv("submission.csv",index = False)